In [ ]:
from PIL import Image
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow.keras as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten,Concatenate
from tensorflow.keras.utils import to_categorical
import os
import pandas as pd
import csv
import cv2
import random
import glob
from IPython.core.debugger import Tracer

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth 
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
download_CT_COVID = drive.CreateFile({'id':'1bDI0Lokr1Do1adDq6kY2I6NIedaPGDbS'})
download_CT_COVID.GetContentFile('clustercovid.zip')

In [ ]:
!unzip /content/clustercovid.zip -d /content/Data

Archive:  /content/clustercovid.zip
   creating: /content/Data/clustercovid/
  inflating: /content/Data/clustercovid/.DS_Store  
  inflating: /content/Data/__MACOSX/clustercovid/._.DS_Store  
   creating: /content/Data/clustercovid/coviddata/
  inflating: /content/Data/clustercovid/firstcovid.py  
  inflating: /content/Data/__MACOSX/clustercovid/._firstcovid.py  
   creating: /content/Data/clustercovid/coviddata/CT_COVID/
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/._CT_COVID  
  inflating: /content/Data/clustercovid/coviddata/.DS_Store  
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/._.DS_Store  
   creating: /content/Data/clustercovid/coviddata/CT_NonCOVID/
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/._CT_NonCOVID  
  inflating: /content/Data/clustercovid/coviddata/CT_COVID/2020.03.01.20029769-p21-73_1%1.png  
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/CT_COVID/._2020.03.01.20029769-p21-73_1%1.png  
  inflating: /content/Data

In [ ]:
download_files = drive.CreateFile({'id':'13VzXjmQ8LE65FtIdtz841w-JvpJ-SYU3'})
download_files.GetContentFile('COVID-CT-masteractual.zip')

In [ ]:
!unzip /content/COVID-CT-masteractual.zip -d /content/textfile

Archive:  /content/COVID-CT-masteractual.zip
0c83254a43230de176489a9b4e3ac12e23b0df53
   creating: /content/textfile/COVID-CT-master/
  inflating: /content/textfile/COVID-CT-master/COVID-CT-MetaInfo.xlsx  
   creating: /content/textfile/COVID-CT-master/Data-split/
   creating: /content/textfile/COVID-CT-master/Data-split/COVID/
  inflating: /content/textfile/COVID-CT-master/Data-split/COVID/testCT_COVID.txt  
  inflating: /content/textfile/COVID-CT-master/Data-split/COVID/trainCT_COVID.txt  
  inflating: /content/textfile/COVID-CT-master/Data-split/COVID/valCT_COVID.txt  
   creating: /content/textfile/COVID-CT-master/Data-split/NonCOVID/
  inflating: /content/textfile/COVID-CT-master/Data-split/NonCOVID/CT_NonCOVID_test_id.csv  
  inflating: /content/textfile/COVID-CT-master/Data-split/NonCOVID/CT_NonCOVID_train_id.csv  
  inflating: /content/textfile/COVID-CT-master/Data-split/NonCOVID/CT_NonCOVID_val_id.csv  
  inflating: /content/textfile/COVID-CT-master/Data-split/NonCOVID/testCT_

In [ ]:
img_dir = "/content/Data/clustercovid/coviddata/CT_COVID"
img_dirnon = "/content/Data/clustercovid/coviddata/CT_NonCOVID"

ct_dir = "/content/textfile/COVID-CT-master/Data-split/COVID/testCT_COVID.txt"
tr_dir = "/content/textfile/COVID-CT-master/Data-split/COVID/trainCT_COVID.txt"
vl_dir = "/content/textfile/COVID-CT-master/Data-split/COVID/valCT_COVID.txt"

nct_dir = "/content/textfile/COVID-CT-master/Data-split/NonCOVID/testCT_NonCOVID.txt"
ntr_dir = "/content/textfile/COVID-CT-master/Data-split/NonCOVID/trainCT_NonCOVID.txt"
nvl_dir = "/content/textfile/COVID-CT-master/Data-split/NonCOVID/valCT_NonCOVID.txt"

CATEGORIES = ["covid", "notcovid"]

In [ ]:
data_path = os.path.join(img_dir, '*g')
data_pathnon = os.path.join(img_dirnon, '*g')

files = glob.glob(data_path)
files = sorted(files, key=str.lower)
filesnon = glob.glob(data_pathnon)
filesnon = sorted(filesnon, key=str.lower)

covidTest = []
covidTestlabel = []

covidTrain = []
covidTrainlabel = []

covidValid = []
covidValidlabel = []

nontest = []
nontestlabel = []

nontrain = []
nontrainlabel = []

nonvalid = []
nonvalidlabel = []

ctFile = open(ct_dir, 'r')
ctLine = ctFile.readlines()

trFile = open(tr_dir, 'r')
tdLine = trFile.readlines()

vlFile = open(vl_dir, 'r')
vlLine = vlFile.readlines()

nonctfile = open(nct_dir, 'r')
nonctline = nonctfile.readlines()

nontrfile = open(ntr_dir, 'r')
nontrline = nontrfile.readlines()

nonvlfile = open(nvl_dir, 'r')
nonvlline = nonvlfile.readlines()

In [ ]:
# changed from 500 to 250 for debug
desired_size = 200

In [ ]:
for f1 in files:
    img = cv2.imread(f1)
    #imgreszie = cv2.resize(img,(480,480))
    old_size = img.shape[:2]
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    img = cv2.resize(img, (new_size[1], new_size[0]))
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top,bottom = delta_h//2, delta_h-(delta_h//2)
    left,right = delta_w//2, delta_w-(delta_w//2)
    color = [0,0,0]
    imgreszie = cv2.copyMakeBorder(img,top,bottom,left,right,cv2.BORDER_CONSTANT,value=color)
    normresize = cv2.normalize(imgreszie, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    f1 = f1[f1.rfind('COVID/') + 6:]
    for line in ctLine:
        if line.replace('\r\n', '').rfind(f1) == 0:
            covidTest.append((np.array(normresize)))
            covidTestlabel.append(1)

    for line in tdLine:
        if line.replace('\r\n', '').rfind(f1) == 0:
            covidTrain.append((np.array(normresize)))
            covidTrainlabel.append(1)


    for line in vlLine:
        if line.replace('\r\n', '').rfind(f1) == 0:
            covidValid.append((np.array(normresize)))
            covidValidlabel.append(1)


In [ ]:
for f1 in filesnon:
    img = cv2.imread(f1)
    #imgreszie = cv2.resize(img, (480, 480))
    old_size = img.shape[:2]
    ratio = float(desired_size) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    img = cv2.resize(img, (new_size[1], new_size[0]))
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    color = [0, 0, 0]
    imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    #print(np.max(imgreszie))
    #print(np.min(imgreszie))
    normresize = (imgreszie* 1.0)/np.max(imgreszie)
    #print(np.max(normresize))
    #print(np.min(normresize))

    #normresize = cv2.normalize(imgreszie,None,alpha=0,beta=1,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_32F)
    origf1 = f1
    f1 = f1[f1.rfind('COVID/') + 6:]
    for line in nonctline:
        if line.replace('\r\n', '').rfind(f1) == 0:
            nontest.append(normresize)
            nontestlabel.append(0)

    for line in nontrline:
        if line.replace('\r\n', '').rfind(f1) == 0:
            nontrain.append(normresize)
            nontrainlabel.append(0)

    for line in nonvlline:
        if line.replace('\r\n', '').rfind(f1) == 0:
            nonvalid.append(normresize)
            nonvalidlabel.append(0)

In [ ]:
combinetraindata = covidTrain + nontrain
#print(len(combinetraindata))
combinetestdata = covidTest + nontest
combinevaliddata = covidValid + nonvalid

combinetrainlabel = covidTrainlabel + nontrainlabel
combinetrainlabel = to_categorical(combinetrainlabel)
combinetestlabel = covidTestlabel + nontestlabel
combinetestlabel = to_categorical(combinetestlabel)
combinevalidlabel = covidValidlabel + nonvalidlabel
combinevalidlabel = to_categorical(combinevalidlabel)

# Randomize the data
rand_order = np.random.permutation(len(combinetraindata))
combinetraindata = np.array(combinetraindata)
combinetraindata = combinetraindata[rand_order, :, :, :]
combinetrainlabel = combinetrainlabel[rand_order]

rand_order = np.random.permutation(len(combinetestdata))
combinetestdata = np.array(combinetestdata)
combinetestdata = combinetestdata[rand_order, :, :, :]
combinetestlabel = combinetestlabel[rand_order]

rand_order = np.random.permutation(len(combinevaliddata))
combinevaliddata = np.array(combinevaliddata)
combinevaliddata = combinevaliddata[rand_order, :, :, :]
combinevalidlabel = combinevalidlabel[rand_order]


In [ ]:
print(np.shape(combinetraindata))
print(np.shape(combinetestdata))
print(np.shape(combinevaliddata))

print(np.shape(combinetrainlabel))
print(np.shape(combinetestlabel))
print(np.shape(combinevalidlabel))


(425, 200, 200, 3)
(203, 200, 200, 3)
(118, 200, 200, 3)
(425, 2)
(203, 2)
(118, 2)


In [ ]:
model = tf.applications.InceptionV3(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(desired_size, desired_size, 3),
    pooling=max,
    classes=2,
    classifier_activation="softmax",
)

In [ ]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 99, 99, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 99, 99, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 99, 99, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
#save weights
checkpoint_filepath = '/content/checkpoint.hdf5'
model_checkpoint_callback = tf.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)


In [ ]:
# complie
optimizer = tf.optimizers.SGD(learning_rate=0.05)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(combinetraindata, combinetrainlabel, batch_size=32, epochs=50,callbacks=[model_checkpoint_callback], verbose=True, validation_data=(combinevaliddata,combinevalidlabel))
model.load_weights(checkpoint_filepath)
loss, accuracy = model.evaluate(combinetestdata, combinetestlabel, verbose=True)
print(f'Test loss: {loss:.3}')
print(f'Test accuracy: {accuracy:.3}')

Epoch 1/50
14/14 [==============================] - 4s 300ms/step - loss: 0.0770 - accuracy: 0.9741 - val_loss: 3.1261 - val_accuracy: 0.5678
Epoch 2/50
14/14 [==============================] - 3s 219ms/step - loss: 0.2162 - accuracy: 0.9435 - val_loss: 1.0262 - val_accuracy: 0.7034
Epoch 3/50
14/14 [==============================] - 3s 228ms/step - loss: 0.3384 - accuracy: 0.8706 - val_loss: 3.2245 - val_accuracy: 0.6864
Epoch 4/50
14/14 [==============================] - 3s 190ms/step - loss: 0.1637 - accuracy: 0.9459 - val_loss: 2.7852 - val_accuracy: 0.4915
Epoch 5/50
14/14 [==============================] - 3s 198ms/step - loss: 0.2501 - accuracy: 0.8988 - val_loss: 2.0187 - val_accuracy: 0.4915
Epoch 6/50
14/14 [==============================] - 3s 191ms/step - loss: 0.1473 - accuracy: 0.9529 - val_loss: 1.5940 - val_accuracy: 0.7034
Epoch 7/50
14/14 [==============================] - 3s 198ms/step - loss: 0.1360 - accuracy: 0.9412 - val_loss: 1.0885 - val_accuracy: 0.6695
Epoch 